<a href="https://colab.research.google.com/github/HarsikaVetrivel/PYTHONAIML/blob/main/MultimodelRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Install dependencies
!pip install transformers langchain sentence-transformers faiss-cpu accelerate --quiet
!pip install torchvision torchaudio --quiet
!pip install git+https://github.com/huggingface/llava.git@main --quiet


  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/huggingface/llava.git /tmp/pip-req-build-ddzo1fbp did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/huggingface/llava.git /tmp/pip-req-build-ddzo1fbp did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [19]:
# 📦 Install required packages
!pip install transformers sentence-transformers faiss-cpu Pillow --quiet

# ✅ Imports
import requests
from PIL import Image
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import os

# 🧠 Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# 📄 Some example documents (text + captions)
docs = [
    "Chest X-ray shows bilateral infiltrates and possible pneumonia.",
    "MRI of the brain reveals no abnormality.",
    "This image contains a cat sitting on a windowsill looking outside.",
    "CT scan shows lung nodules suggestive of metastasis."
]
doc_embeddings = embedder.encode(docs)

# 🧠 Create FAISS index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

# 🧠 Text QA model (Free, no API key, no torch!)
qa_model = pipeline("text2text-generation", model="google/flan-t5-base")

# 🖼️ Function to generate image caption (simulated)
def fake_image_caption(image):
    # You can replace this with real captioning later
    return "An X-ray image showing abnormalities in the lungs"

# 🧠 Main RAG function (Image + Text → QA)
def multimodal_rag(image_path, question):
    # Step 1: Get a caption from the image
    caption = fake_image_caption(image_path)
    print("🖼️ Image Caption:", caption)

    # Step 2: Retrieve the most relevant context
    query_embedding = embedder.encode([caption + " " + question])
    D, I = index.search(np.array(query_embedding), k=1)
    retrieved = docs[I[0][0]]
    print("🔍 Retrieved Context:", retrieved)

    # Step 3: Build prompt
    prompt = f"Context: {retrieved}\nQuestion: {question}\nAnswer:"

    # Step 4: Use FLAN-T5 to answer
    result = qa_model(prompt, max_new_tokens=100)[0]["generated_text"]
    return result

# 📸 Test image
img_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/image_classification.jpeg"
img_path = "test.jpg"
with open(img_path, "wb") as f:
    f.write(requests.get(img_url).content)

# 🎯 Run the multimodal RAG
question = "What condition does the scan indicate?"
answer = multimodal_rag(img_path, question)

print("\n🧠 Final Answer:", answer)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


🖼️ Image Caption: An X-ray image showing abnormalities in the lungs
🔍 Retrieved Context: Chest X-ray shows bilateral infiltrates and possible pneumonia.

🧠 Final Answer: (iii)
